In [1]:
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt

import nibabel as nib

import torch
import torch.optim as optim

from torchsummary import summary

%matplotlib inline

They have only one size.

In [2]:
# set(nib.load(b).get_fdata().shape for b in glob('blurred/*.BRIK'))

## Model

In [3]:
from model import *

In [4]:
model = DeepBrain(inplanes=179, planes=3)

G:\My Drive\termpj\model.py:89: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(m.weight, gain=nn.init.calculate_gain('relu'))


In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(summary(model, input_size=(179, 72, 72, 36)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1        [-1, 3, 72, 72, 36]             540
       BatchNorm3d-2        [-1, 3, 72, 72, 36]               6
              ReLU-3        [-1, 3, 72, 72, 36]               0
            Conv3d-4       [-1, 16, 36, 36, 18]           1,312
       BatchNorm3d-5       [-1, 16, 36, 36, 18]              32
              ReLU-6       [-1, 16, 36, 36, 18]               0
            Conv3d-7       [-1, 32, 36, 36, 18]             544
       BatchNorm3d-8       [-1, 32, 36, 36, 18]              64
            Conv3d-9       [-1, 32, 36, 36, 18]          13,856
      BatchNorm3d-10       [-1, 32, 36, 36, 18]              64
             ReLU-11       [-1, 32, 36, 36, 18]               0
           Conv3d-12       [-1, 32, 36, 36, 18]          27,680
      BatchNorm3d-13       [-1, 32, 36, 36, 18]              64
             ReLU-14       [-1, 32, 36,

## DataLoader

In [6]:
from torch.utils.data import DataLoader, Dataset

class MyDataset(Dataset):
    
    def __init__(self, datatype='blur', test=False):
        
        if datatype == 'blur':
            self.datafiles = glob('blurred/*.BRIK')
            
        elif datatype == 'scale':
            self.datafiles = glob('scaled/*.BRIK')
            
        else:
            pass
    
        if test: self.datafiles[-5:]
        else: self.datafiles[:-5]
        self.demographics = pd.read_csv('participants_with_runs.csv')
        
    def __getitem__(self, idx):
        
        x = torch.FloatTensor(nib.load(self.datafiles[idx]).get_fdata())
        y = torch.tensor(self.demographics.loc[idx].Age > 50)
        
        return x, y
    
    def __len__(self):
        return len(self.datafiles)

In [7]:
dset = MyDataset()
dloader = DataLoader(dset, batch_size=8)

## Run

In [8]:
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

In [9]:
for e in range(1):
    
    for i, (x, y) in enumerate(dloader):
        
        x, y = x.permute(0, 4, 1, 2, 3).to(device), y.to(device)
        
        optimizer.zero_grad()
        
        y_pred = model.forward(x).to(device)
        
        loss = loss_fn(y_pred.squeeze(1), y)
        
        loss.backward()
        optimizer.step()

C:\Users\pha\anaconda3\lib\site-packages\torch\nn\modules\loss.py:516: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([3240])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


ValueError: Target and input must have the same number of elements. target nelement (8) != input nelement (3240)

In [10]:
y

tensor([False, False, False, False, False, False,  True,  True],
       device='cuda:0')

In [11]:
y_pred

tensor([[0.5005],
        [0.5009],
        [0.5014],
        ...,
        [0.5032],
        [0.4982],
        [0.4994]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [13]:
model(x).shape

RuntimeError: CUDA out of memory. Tried to allocate 1020.00 MiB (GPU 0; 8.00 GiB total capacity; 3.65 GiB already allocated; 0 bytes free; 3.68 GiB reserved in total by PyTorch)